# Vector Similarity
1. Manhattan distance L1
2. Euclidean distance L2
3. Cosine distance
4. Dot product distance

In [2]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
os.environ['USER_AGENT'] = 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    separators=["\n\n", "\n"]
)

In [15]:
import requests
from collections import deque
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pathlib
import os
import string
from langchain.chains import create_extraction_chain


Getting content from https://www.niaid.nih.gov/
['healthy volunteers help niaid evaluate treatment prevention options wide range diseases conditions mission areas help us improve human health healthy volunteers help niaid evaluate treatment prevention options wide range diseases conditions mission areas help us improve human health niaid mobilized intramural extramural-supported research infrastructure provide scientific support u.s. department agriculture usda department health human services hhs niaid mobilized intramural extramural-supported research infrastructure provide scientific support u.s. department agriculture usda department health human services hhs niaid making every effort eliminate use stigmatizing terminology advance use person-first inclusive respectful language niaid hiv language guide relevance beyond hiv includes language related areas research diseases conditions gender sexuality general research terminology niaid making every effort eliminate use stigmatizing te

In [6]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [135]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
import requests

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(['h1', "article"])
header_template = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
loader = WebBaseLoader(
    web_paths=("https://www.niaid.nih.gov/",),
    bs_kwargs={"parse_only": bs4_strainer},
)

scraped_doc = loader.scrape()
docs = loader.load_and_split(text_splitter)

[]

In [146]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/db/'
!rm -rf ./docs/chroma

In [ ]:
vectordb = Chroma.from_documents(
    documents=[],
    embedding=embedding,
    persist_directory=persist_directory
)


In [127]:
print(vectordb._collection.count())

5


In [176]:
type(vectordb)


langchain_community.vectorstores.chroma.Chroma

In [133]:
question='who is Jeanne Marrazzo'
docs = vectordb.similarity_search(question,k=1)
print(len(docs))

docs[0]

1


Document(metadata={'source': 'https://www.niaid.nih.gov/'}, page_content="Director\n\n                    Jeanne Marrazzo, M.D., M.P.H.\n                \n\n\n\n                    Dr. Marrazzo began her tenure as the sixth NIAID Director in the fall of 2023.  She oversees a $6.6 billion budget that supports research to advance the understanding, diagnosis, treatment and prevention of infectious, immunologic, and allergic diseases. She is internationally recognized for her research and education efforts in the field of sexually transmitted infections, especially as they affect women’s health.\n                \nLearn more about Dr. Marrazzo\n\n\n\n\nNewsroom\n\n\n\n\nVisit the NIAID Newsroom\n\n\n\n\n\nRead NIAID's Latest News Releases\n\n\n\n\n\nContact the NIAID Media Team\n\n\n\n\n\nFollow NIAID on X \n\n\n\n\n\nSubscribe to NIAID on YouTube \n\n\n\n\n\n\n\n\nFeatured Areas of Research\n\n\n\n\nAntimicrobial Resistance\n\n\nAntimicrobial Resistance Research Initiatives  \nNIAID-Supp

In [5]:
vectorstore = Chroma("langchain_store", embedding)


NameError: name 'Chroma' is not defined

In [154]:
def web_scraping_dfs(url, limit = -1):
    root = '/'
    nodes_list = deque() # stack
    visited = set()      # set

    # add root
    nodes_list.append(root)

    # title and content only
    bs4_strainer = bs4.SoupStrainer(['h1', "article"])
    
    # set counter to 0
    counter = 0
    while len(nodes_list) > 0 and (counter < limit or limit == -1):
        counter = counter + 1
        node=nodes_list.pop()
        visited.add(node)
        full_node_url = url + node
        print('Getting content from', full_node_url)
        loader = WebBaseLoader(
            web_paths=(full_node_url,),
            bs_kwargs={"parse_only": bs4_strainer},
        )
        loader.session.headers['User-Agent'] = 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'

        # load and tokenize
        docs = loader.load_and_split(text_splitter)
        # to find all a tags
        scraped_doc = loader.scrape()
        # embed the docs
        vectorstore.add_documents(docs)
        # update nodes_list using anchor tags
        for a in scraped_doc.findAll('a'):
            # make sure the href exists and is internal
            if(not a.has_attr('href') or not a['href'].startswith('/')):
                continue
            # exclude files
            url_path = urlparse(a['href']).path
            file_extension = os.path.splitext(url_path)[1]
            if(file_extension in files_extension_to_exclude):
                continue
            # maintain stack
            if(a['href'] not in visited and a['href'] not in nodes_list):
                nodes_list.append(a['href'])


/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [155]:
web_scraping_dfs("https://www.niaid.nih.gov", 3)
vectorstore.persist()

Getting content from https://www.niaid.nih.gov/
Getting content from https://www.niaid.nih.gov/node/4220
Getting content from https://www.niaid.nih.gov/grants-contracts/preparing-foreign-organization-system-review


In [156]:
vectorstore._collection.count()

13

In [159]:
question='who is Jeanne Marrazzo'
docs = vectordb.similarity_search(question,k=1)
print(len(docs))

preprocess_text(docs[0].page_content)

1


"director jeanne marrazzo m.d. m.p.h dr. marrazzo began tenure sixth niaid director fall 2023. oversees 6.6 billion budget supports research advance understanding diagnosis treatment prevention infectious immunologic allergic diseases internationally recognized research education efforts field sexually transmitted infections especially affect womens health learn dr. marrazzo newsroom visit niaid newsroom read niaid 's latest news releases contact niaid media team follow niaid x subscribe niaid youtube featured areas research antimicrobial resistance antimicrobial resistance research initiatives niaid-supported research antimicrobial resistance antimicrobial resistance threats hiv/aids"

In [160]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [161]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [187]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [226]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=False,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

#### result = qa_chain({"query": question})
result

In [228]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

In [235]:
def create_vector_store():
    persist_directory = 'docs/db/'
    !rm -rf ./docs/db
    vectorstore = Chroma("langchain_store", embedding)
    return vectorstore


In [230]:
def create_qa_chain(vectorstore:Chroma, chain_type, prompt, k):
    # define retriever
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=False,
            
        return_generated_question=True,
        combine_docs_chain_kwargs={"prompt": prompt}
    )
    return qa_chain

In [231]:
def load_pdf(file, chain_type, k):
    # create store
    vectorstore = create_vector_store()
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # load documents
    vectorstore.add_documents(docs)
    return vectorstore
    

In [6]:
files_extension_to_exclude = ['.pdf', '.doc', '.docx', '.xls', '.xlsx',
                              'jpg', '.jpeg', '.png', '.gif', '.ico',
                              '.svg', 'webp']
def load_web(file, chain_type, k):
    root = '/'
    nodes_list = deque() # stack
    visited = set()      # set

    # add root
    nodes_list.append(root)

    # title and content only
    bs4_strainer = bs4.SoupStrainer(['h1', "article"])
    
    # set counter to 0
    counter = 0
    while len(nodes_list) > 0 and (counter < limit or limit == -1):
        counter = counter + 1
        node=nodes_list.pop()
        visited.add(node)
        full_node_url = url + node
        print('Getting content from', full_node_url)
        loader = WebBaseLoader(
            web_paths=(full_node_url,),
            bs_kwargs={"parse_only": bs4_strainer},
        )
        loader.session.headers['User-Agent'] = 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'

        # load and tokenize
        docs = loader.load_and_split(text_splitter)
        # to find all a tags
        scraped_doc = loader.scrape()
        # embed the docs
        vectorstore.add_documents(docs)
        # update nodes_list using anchor tags
        for a in scraped_doc.findAll('a'):
            # make sure the href exists and is internal
            if(not a.has_attr('href') or not a['href'].startswith('/')):
                continue
            # exclude files
            url_path = urlparse(a['href']).path
            file_extension = os.path.splitext(url_path)[1]
            if(file_extension in files_extension_to_exclude):
                continue
            # maintain stack
            if(a['href'] not in visited and a['href'] not in nodes_list):
                nodes_list.append(a['href'])

In [233]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

def create_prompt():
    # Build prompt
    template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
    {context}
    Question: {question}
    Helpful Answer:"""
    prompt = PromptTemplate.from_template(template)
    return prompt

In [224]:
import gradio as gr
import random
import time
from itertools import chain

# create a prompt
prompt = create_prompt()
# create a question-answer chain
qa_chain = create_qa_chain(vectordb, "stuff", prompt, 3)
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    history = []
    def respond(message, chat_history):
        bot_message = qa_chain({"question": message, "chat_history": history})
        print(bot_message)
        history.extend([(message, bot_message['answer'])])
        chat_history.append((message, bot_message['answer']))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(inline=True)

Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.


{'question': 'hi', 'chat_history': [], 'answer': "I'm sorry, but there is no question provided in the context. Thanks for asking!", 'generated_question': 'hi'}
{'question': 'hello', 'chat_history': [('hi', "I'm sorry, but there is no question provided in the context. Thanks for asking!")], 'answer': "I can help you with information on volunteering for clinical studies, NIAID's research agenda for 2024, and the NIAID HIV Language Guide. Thanks for asking!", 'generated_question': 'What can I help you with today?'}
